# Проект по SQL

**ЦЕЛЬ:** проанализировать базу данных сервиса для чтения книг 
    
**Задачи:**подготовить анализ о ĸоличестве ĸниг, отзывах, популярных изданиях и авторах

**Ход работы**
1. Исследование таблиц
2. Посчитать, сколько книг вышло после 1 января 2000 года;
3. Для каждой книги посчитайте количество обзоров и среднюю оценку;
4. Определить издательство, которое выпустило наибольшее число книг толще 50 страниц;
5. Определить автора с самой высокой средней оценкой книг(учитываются только книги с 50 и более оценками);
6. Посчитать среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

**Описание данных**

*Таблица books*

Содержит данные о книгах:

* book_id — идентификатор книги;
* author_id — идентификатор автора;
* title — название книги;
* num_pages — количество страниц;
* publication_date — дата публикации книги;
* publisher_id — идентификатор издателя.

*Таблица authors*

Содержит данные об авторах:

* author_id — идентификатор автора;
* author — имя автора.

*Таблица publishers*

Содержит данные об издательствах:

* publisher_id — идентификатор издательства;
* publisher — название издательства.

*Таблица ratings*

Содержит данные о пользовательских оценках книг:

* rating_id — идентификатор оценки;
* book_id — идентификатор книги;
* username — имя пользователя, оставившего оценку;
* rating — оценка книги.

*Таблица reviews*

Содержит данные о пользовательских обзорах:

* review_id — идентификатор обзора;
* book_id — идентификатор книги;
* username — имя автора обзора;
* text — текст обзора.

## Исследование таблиц

In [1]:
import pandas as pd
from sqlalchemy import text, create_engine

In [2]:
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

In [3]:
engine = create_engine(connection_string, connect_args={'sslmode':'require'})


In [4]:
def all_table(table):
    query = '''SELECT * FROM {} LIMIT 5'''.format(table)
    df = pd.io.sql.read_sql(query, con = engine)
    display (df)

In [5]:
list = ['books','authors', 'ratings', 'reviews', 'publishers']
for i in list:

    print('\033[1m' + "Таблица", i)
    all_table(i)

Таблица books


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


Таблица authors


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


Таблица ratings


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


Таблица reviews


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


Таблица publishers


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


## Cколько книг вышло после 1 января 2000 года

In [6]:
books_count = '''
SELECT 
    COUNT(book_id)
    FROM books 
    WHERE publication_date >= '2000-01-02' '''
pd.io.sql.read_sql(books_count, con = engine)

,count
0,819


**После 1 января 2000 года вышло 819 книг**

## Количество обзоров и средняя оценку для каждой книги

In [7]:
review_rating= '''
SELECT 
    books.title AS title,
    books.book_id AS id,
    COUNT(DISTINCT reviews.text) AS count_reviews,
    ROUND(AVG(ratings.rating),1) AS avg_rating
FROM books
INNER JOIN ratings ON ratings.book_id = books.book_id
INNER JOIN reviews ON reviews.book_id = books.book_id
GROUP BY books.title, books.book_id
ORDER BY count_reviews  DESC 
'''
pd.io.sql.read_sql(review_rating, con = engine)

,title,id,count_reviews,avg_rating
0,Twilight (Twilight #1),948,7,3.7
1,The Road,854,6,3.8
2,The Book Thief,656,6,4.3
3,The Glass Castle,734,6,4.2
4,Water for Elephants,963,6,4.0
...,...,...,...,...
989,Naked Empire (Sword of Truth #8),465,1,3.5
990,Moo Baa La La La!,446,1,3.0
991,Merrick (The Vampire Chronicles #7),431,1,4.0
992,Babyville,92,1,3.5


## Издательство, которое выпустило наибольшее число книг толще 50 страниц 

In [8]:
publishers_50 = '''
SELECT 
    publishers.publisher AS name, 
    COUNT(books.book_id) AS count_books
FROM publishers 
INNER JOIN books ON books.publisher_id = publishers.publisher_id
WHERE num_pages > 50
GROUP BY name
ORDER BY count_books DESC
LIMIT 1
'''
pd.io.sql.read_sql(publishers_50, con = engine)

,name,count_books
0,Penguin Books,42


**Издательство, которое выпустило наибольшее число книг толще 50 страниц - Penguin Books**

## Автор с самой высокой средней оценкой книг

In [9]:
best_autor = '''
SELECT authors.author,
       ROUND(AVG(ratings.rating),2) AS avg_rating
FROM authors
INNER JOIN books ON books.author_id = authors.author_id
INNER JOIN ratings ON ratings.book_id = books.book_id
WHERE books.book_id IN
    (SELECT books.book_id
     FROM books
     INNER JOIN ratings ON ratings.book_id = books.book_id
     GROUP BY books.book_id
     HAVING COUNT(ratings.rating_id)>=50)
GROUP BY authors.author
ORDER BY avg_rating DESC

LIMIT 1
'''
pd.io.sql.read_sql(best_autor, con = engine)

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.29


**Авторки с самой высокой средней оценкой книг - Джоан Роулинг/Мэри Грандпре**

##  Cреднее количество обзоров от пользователей, которые поставили больше 48 оценок

In [10]:
user_review = '''
SELECT
    AVG(u.count_reviews)
FROM
(SELECT 
    username,
    COUNT(review_id) AS count_reviews
 FROM reviews
 WHERE username IN 
 (SELECT 
    username
FROM ratings
GROUP BY username
HAVING COUNT(rating_id)>48)
GROUP BY username) AS u
'''
pd.io.sql.read_sql(user_review, con = engine)

,avg
0,24.0


**Cреднее количество обзоров от пользователей, которые поставили больше 48 оценок- 24**